In [1]:
import os 
from dotenv import load_dotenv 
from google import genai 
from google.genai import types 
import json 

In [20]:
# --- Load Env --- 
load_dotenv()
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

# --- Initialize Gemini Client --- 
client = genai.Client(api_key=GEMINI_API_KEY)

# --- Model Constants --- 
MODEL_FLASH = "gemini-3-flash-preview"
MODEL_PRO = "gemini-3-pro-preview" 

In [21]:
# --- Standard Mode : Simple Question --- 
response = client.models.generate_content(
    model=MODEL_FLASH,
    contents = "What is the capital of France ? "
)
print("--- Standard Response ---")
print(response.text)

--- Standard Response ---
The capital of France is **Paris**.


In [22]:
# --- Standard Mode : Streaming --- 
print("---- Streanming Response : ----")
stream = client.models.generate_content_stream(
    model=MODEL_FLASH,
    contents="Explain quantum computing in 10 sentences."
)
full_response = ""
for chunk in stream:
    if chunk.text:
        print(chunk.text, end="", flush=True)
        full_response += chunk.text
print("\n\n Strem Complete")


---- Streanming Response : ----
Quantum computing is a revolutionary field of technology that uses the principles of quantum mechanics to process information. Unlike classical computers that use bits as either a 0 or a 1, quantum computers use "qubits." Through a property called superposition, a qubit can exist in multiple states simultaneously, representing both 0 and 1 at the same time. Another core principle is entanglement, where qubits become interconnected so that the state of one instantly influences the state of another, even across long distances. These unique behaviors allow quantum computers to perform complex calculations at speeds impossible for traditional machines. While a standard computer processes tasks sequentially, a quantum computer can explore millions of possibilities at once. This makes them ideal for solving problems like simulating molecular interactions for new drug discovery or optimizing global logistics. They also pose a significant challenge to cybersecur

In [7]:
# --- Standard Mode : System Prompt For Warm AI ---
system_prompt = """
You are Warm AI, an intelligent assistant specializing in 
professional networking and career insights. You provide helpful, accurate, 
and friendly responses about careers, companies, and professional development.
"""
response = client.models.generate_content(
    model=MODEL_FLASH,
    contents="How can I improve my LinkedIn Profile?",
    config=types.GenerateContentConfig(
        system_instruction=system_prompt,
        temperature=0.7,
        max_output_tokens=500
    )
)
print("Response With System Prompt : ")
print(response.text)

Response With System Prompt : 
Hello! I’m **Warm AI**, and I’d be delighted to help you polish your LinkedIn profile. Think of your profile as your 24/7 digital billboard—it’s often the first impression you make on recruiters and peers.

To make your profile truly stand out, here is a step-by-step guide to elevating your professional brand:

### 1. Master Your Visuals
*   **The Profile Photo:** Use a high-quality, well-lit headshot where you look approachable and professional. A friendly smile goes a


In [23]:
# --- Web Search Mode : Grounded With Google Search ---
response = client.models.generate_content(
    model=MODEL_FLASH,
    contents="What are the latest AI Startup funding rounds in 2025?",
    config=types.GenerateContentConfig(
        tools=[types.Tool(google_search=types.GoogleSearch())],
        temperature=0.3
    )
)
print("--- Grounded Response ---")
print(response.text)

ClientError: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. ', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limits'}]}]}}

In [18]:
# --- Extract Citations from Grounded Response ---
def extract_grounding_sources(response):
    """
        Extract source URLs from grounding metadata
    """
    sources = []
    
    if hasattr(response, 'candidates') and response.candidates:
        candidate = response.candidates[0]
        
        # Check for grounding metadata
        if hasattr(candidate, 'grounding_metadata') and candidate.grounding_metadata:
            metadata = candidate.grounding_metadata
            
            # Extract grounding chunks (sources)
            if hasattr(metadata, 'grounding_chunks') and metadata.grounding_chunks:
                for chunk in metadata.grounding_chunks:
                    if hasattr(chunk, 'web') and chunk.web:
                        sources.append({
                            "title": chunk.web.title if hasattr(chunk.web, 'title') else "Unknown",
                            "url": chunk.web.uri if hasattr(chunk.web, 'uri') else None
                        })
    
    return sources

# --- Test Grounded Search ---
grounded_response = client.models.generate_content(
    model=MODEL_FLASH,
    contents="What is the current market cap of NVIDIA in 2025?",
    config=types.GenerateContentConfig(
        tools=[types.Tool(google_search=types.GoogleSearch())]
    )
)

print(" --- Grounded Answer: ---")
print(grounded_response.text)
print("\n --- Sources: ---")
sources = extract_grounding_sources(grounded_response)
for src in sources:
    print(f"  - {src['title']}: {src['url']}")

 --- Grounded Answer: ---
The market capitalization of NVIDIA as of December 19, 2025, is approximately in the range of **$4.23 Trillion to $4.4 Trillion**.

For instance, specific figures reported around this date include:
*   **$4.4 trillion** as of December 19, 2025.
*   **$4.36 trillion** as of December 19, 2025.
*   **$4.232 trillion** as of December 19, 2025.

Market capitalization is calculated by multiplying the current stock price by the number of shares outstanding, and it fluctuates daily with the stock market.

 --- Sources: ---
  - macrotrends.net: https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGsmAXTQ6xpD2EjBE-M8IPGb7mP_9pUOR37vW_d8DVPZSNT891EdChxSZKiQontDhd7389RwqRdEbTgiPYu-FJ7YvFEVgrnACM0m8EDDelAqAMBLHspzyOLCBPqszfqVqJcy6EiACtXNxkQj6IPLRjhB9ZlTXwALLiXQ9I=
  - capital.com: https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQE0ui3akbrUZYtVISvt9J1BJR80xSH0jiVN0DmX9rs3gTSRnoRvHOL4ie0xDL4hmcVCFjZVa_7SKe7A9GBtZ0NPPFCrhK9Dk0AMleefGFuiQGee